### GET Soil Moister Image Collection

Dataset Used: https://developers.google.com/earth-engine/datasets/catalog/NASA_SMAP_SPL3SMP_E_005

### IMPORTS

In [1]:
import time
from typing import Any

import ee
import geemap
import geopandas as gpd
import pandas as pd
from tqdm import tqdm

### AUTHENTICATE AND INITIALIZE

In [2]:
# Authenticate to the Earth Engine servers
ee.Authenticate()

# Initialize the Earth Engine module.
ee.Initialize()

### VARIABLES

In [3]:
DATASET_NAME: str = "NASA/SMAP/SPL3SMP_E/005"
START_DATE: str = "2023-01-01"
END_DATE: str = "2023-12-04"
BANDS: list[str] = ["soil_moisture_am", "soil_moisture_pm"]
SCALE: int = 9000
# ZONES: range = range(2, 10)
TARGET_ZONE = 9
ZONES = [4, 9]
MIN = 0
MAX = 0.5
UNITS = "Volume fraction"
MAXPIXELS = 1e10

### GET IMAGES

In [4]:
shapefile_path = f"/vsizip/../shape_files/zone_{TARGET_ZONE}.zip/layers/POLYGON.shp"
gdf = gpd.read_file(shapefile_path)
roi = geemap.gdf_to_ee(gdf)

In [5]:
# roi.geometry()

In [6]:
SMAP = ee.ImageCollection(DATASET_NAME)

SMAP_ROI = SMAP.filter(ee.Filter.date(START_DATE, END_DATE)).filter(ee.Filter.bounds(roi.geometry())) #.median()

MEDIAN = SMAP_ROI.median()

In [7]:
# Export with 9000m resolution (refer to dataset)
image_ids = SMAP_ROI.aggregate_array("system:index").getInfo()
print("Total Images: ", len(image_ids))

Total Images:  334


### VISUALIZATION

In [8]:
from IPython.display import display, HTML

palette_lst = ["0300ff", "418504", "efff07", "efff07", "ff0303"]

visualization = {
    "bands": BANDS[0],
    "min": MIN,
    "max": MAX,
    "palette": palette_lst
}

# Set opacity level between 0 (transparent) and 1 (opaque)
opacity_level = 0.4  
# Draw boundaries of the shapefile region loaded in 'table'
boundary = ee.FeatureCollection(roi)

Map = geemap.Map()
# m.set_center(9.188540, 45.464664, 9)
# Map.set_center(9.143192990831077, 45.53639619976871, 10)
Map.centerObject(roi.geometry(), 13)
Map.add_layer(MEDIAN.clip(roi.geometry()), visualization, BANDS[0])
Map.addLayer(boundary, {"color": "000000"}, "Region Boundary", True, opacity_level)
Map.addLayerControl()

# Legend setup
min_value = MIN  # Minimum Value
max_value = MAX   # Maximum Value
num_colors = len(palette_lst)
interval = (max_value - min_value) / num_colors
ranges = [f"{min_value + i * interval:.2f} to {min_value + (i + 1) * interval:.2f} ({UNITS})" for i in range(num_colors)]
legend_dict = dict(zip(ranges, ["#" + color for color in palette_lst]))

# Display a title above the map
display(HTML(f"<h3>{BANDS[0]}: Milan-Italy-Zone {TARGET_ZONE}</h3>"))
Map.add_legend(title=f"{BANDS[0]} ({UNITS}) Milan-Zone {TARGET_ZONE}", legend_dict=legend_dict)

Map

Map(center=[45.512724879429385, 9.180956342696671], controls=(WidgetControl(options=['position', 'transparent_…

### DOWNLOAD IMAGES

In [9]:
for zone in tqdm(ZONES):
    shapefile_path = f"/vsizip/../shape_files/zone_{zone}.zip/layers/POLYGON.shp"
    gdf = gpd.read_file(shapefile_path)
    roi = geemap.gdf_to_ee(gdf)

    SMAP = ee.ImageCollection(DATASET_NAME)

    SMAP_ROI = SMAP.filter(ee.Filter.date(START_DATE, END_DATE)).filter(ee.Filter.bounds(roi.geometry())) #.median()

    # Median
    MEDIAN = SMAP_ROI.median()

    # EXPORTING THE MEDIAN IMAGE
    task_median = ee.batch.Export.image.toDrive(
        image=MEDIAN.select(BANDS),  # Bands
        description=f"Median_Image_Zone{zone}",
        fileNamePrefix=f"zone{zone}_median_sm_{START_DATE}_{END_DATE}",
        folder=f"sm_roi_median_9km_{END_DATE}",
        scale=SCALE,
        region=roi.geometry().getInfo()["coordinates"],
        maxPixels=MAXPIXELS
    )
    
    task_median.start()
    while task_median.active():
        print("Polling for task (id: {}).".format(task_median.id))
        time.sleep(5)  # Poll every 5 seconds

    
    print(f"Task for Zone {zone} median image download completed with state {task_median.status()['state']}: ")

    # EXPORTING ALL IMAGES WITH 30m RESOLUTION (refer to dataset)
    image_ids = SMAP_ROI.aggregate_array("system:index").getInfo()
    print("#" * 50)
    print("Total Images: ", len(image_ids))
    print("#" * 50)

    for i, image_id in enumerate(tqdm(image_ids, desc="Exporting Images")):
        image = ee.Image(SMAP_ROI.filter(ee.Filter.eq("system:index", image_id)).first())
        task = ee.batch.Export.image.toDrive(
        **{
            "image": image.select(BANDS),
            "description": "Image Export {}".format(i + 1),
            "fileNamePrefix": f"zone{zone}_sm_{image.id().getInfo()}",
            "folder": f"sm_roi_9km_{END_DATE}",
            "scale": SCALE,
            "region": roi.geometry().getInfo()["coordinates"], # Use ROI geometry
            # "region": image.geometry().bounds().getInfo()["coordinates"],
            "maxPixels": MAXPIXELS,
        })
        task.start()
        # Monitor the task
        while task.active():
            print("Polling for task (id: {}).".format(task.id))
            time.sleep(5)  # Poll every 5 seconds
        
        print(f"Task {i + 1} completed with state {task.status()['state']}: ")    

  0%|          | 0/2 [00:00<?, ?it/s]

Polling for task (id: ZMUZTUC7PBO3MKR3CQHGHUA5).
Polling for task (id: ZMUZTUC7PBO3MKR3CQHGHUA5).
Polling for task (id: ZMUZTUC7PBO3MKR3CQHGHUA5).
Polling for task (id: ZMUZTUC7PBO3MKR3CQHGHUA5).
Polling for task (id: ZMUZTUC7PBO3MKR3CQHGHUA5).
Task for Zone 4 median image download completed with state COMPLETED: 
##################################################
Total Images:  334
##################################################



Exporting Images:   0%|          | 0/334 [00:00<?, ?it/s]

Polling for task (id: JWURTRUAB5C5G4VMEMWJCRUO).
Polling for task (id: JWURTRUAB5C5G4VMEMWJCRUO).
Polling for task (id: JWURTRUAB5C5G4VMEMWJCRUO).



Exporting Images:   0%|          | 1/334 [00:18<1:41:31, 18.29s/it]

...

Task 332 completed with state COMPLETED: 
Polling for task (id: F4J425LZCIWLTOKXOCE2FGAV).
Polling for task (id: F4J425LZCIWLTOKXOCE2FGAV).
Polling for task (id: F4J425LZCIWLTOKXOCE2FGAV).
Polling for task (id: F4J425LZCIWLTOKXOCE2FGAV).



Exporting Images: 100%|█████████▉| 333/334 [1:58:01<00:22, 22.31s/it]

Task 333 completed with state COMPLETED: 
Polling for task (id: XDCGU7ATQ6EQGL6RWIKYCXQU).
Polling for task (id: XDCGU7ATQ6EQGL6RWIKYCXQU).
Polling for task (id: XDCGU7ATQ6EQGL6RWIKYCXQU).



100%|██████████| 2/2 [3:47:40<00:00, 6830.37s/it]  

Task 334 completed with state COMPLETED: 
